In [21]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../')

from sqlalchemy import create_engine, text

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA

USER_DB = POSTGRES_UTEA['USER']
PASS_DB = POSTGRES_UTEA['PASSWORD']
HOST_DB = POSTGRES_UTEA['HOST']
PORT_DB = POSTGRES_UTEA['PORT']
NAME_DB = POSTGRES_UTEA['DATABASE']

ENGINE = create_engine(f'postgresql+psycopg://{USER_DB}:{PASS_DB}@{HOST_DB}:{PORT_DB}/{NAME_DB}')

In [22]:
#path_xlsx_avance = r'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/AVANCE DE COSECHA V1.xlsx'
path_xlsx_avance = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE SEMANAL\AVANCE DE COSECHA V1.xlsx'

In [23]:
xlsx_resumen = pd.read_excel(path_xlsx_avance, sheet_name='AVANCE_FECHAS')

In [24]:
# Convertir a número (los no válidos se vuelven NaN)
xlsx_resumen['COD_COS'] = pd.to_numeric(xlsx_resumen['COD_COS'], errors='coerce')
# Eliminar filas con NaN
xlsx_resumen = xlsx_resumen.dropna(subset=['COD_COS'])
# Convertir a int
xlsx_resumen['COD_COS'] = xlsx_resumen['COD_COS'].astype(int)
xlsx_resumen

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGADO
0,1,2025-06-05 00:00:00,12.28,893.11,815.18
1,1,2025-06-10 00:00:00,NaN,NaN,1958.91
2,1,2025-06-13 00:00:00,NaN,NaN,774.88
3,1,2025-06-16 00:00:00,NaN,NaN,915.85
4,1,2025-06-18 00:00:00,65.49,3642.64,650.08
...,...,...,...,...,...
8799,636,(en blanco),2.89,0.00,NaN
8800,637,2025-08-27 00:00:00,6.70,312.90,287.19
8801,637,2025-08-29 00:00:00,1.67,72.22,219.52
8802,637,2025-09-03 00:00:00,8.59,359.22,542.94


In [25]:
# 1. Intentar convertir a fecha (los no válidos quedan como NaT)
xlsx_resumen['FECHA'] = pd.to_datetime(xlsx_resumen['FECHA'], errors='coerce')
# Eliminar filas con fechas inválidas
xlsx_resumen = xlsx_resumen.dropna(subset=['FECHA'])

In [26]:
xlsx_resumen[xlsx_resumen['COD_COS'] == 45]

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGADO
680,45,2025-06-13,NaN,NaN,25.81
681,45,2025-06-16,NaN,NaN,139.89
682,45,2025-06-18,NaN,NaN,139.78
683,45,2025-06-22,6.03,506.17,304.08
684,45,2025-08-02,1.11,72.00,1548.56
685,45,2025-08-09,68.58,5765.18,6003.34
686,45,2025-08-11,22.27,1726.55,1791.97
687,45,2025-08-14,20.38,1615.68,2183.57
688,45,2025-08-19,31.95,2915.56,4674.00
689,45,2025-08-27,43.85,4071.14,7255.71


In [27]:
def colapsar_tabla(df_param):
    suma_acumulador = [None] * len(df_param)
    acumulador = 0
    contador = 0
    for i, r in df_param.iterrows():
        if(r['AREA'] == 0):
            acumulador += r['ENTREGADO']
            contador += 1
        else:
            acumulador += r['ENTREGADO']
            suma_acumulador[contador] = acumulador
            acumulador = 0
            contador += 1
            continue
    if suma_acumulador[-1] == None:
        suma_acumulador[-1] = acumulador

    df_param['ENTREGAS'] = suma_acumulador
    
    df_param = df_param.dropna(subset=['ENTREGAS'])
    df_param = df_param.drop(columns=['ENTREGADO'])
    #df_param = df_param[df_param['AREA']!=0]
    return df_param

In [28]:
list_cod_cos = list(set(xlsx_resumen['COD_COS']))

In [29]:
list_df = []

In [30]:
for i in list_cod_cos:
    grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
    result = colapsar_tabla(grupo)
    list_df.append(result)

In [31]:
df_combinado = pd.concat(list_df, ignore_index=True)

In [32]:
df_combinado[df_combinado['COD_COS'] == 45]

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGAS
467,45,2025-06-22,6.03,506.17,609.56
468,45,2025-08-02,1.11,72.00,1548.56
469,45,2025-08-09,68.58,5765.18,6003.34
470,45,2025-08-11,22.27,1726.55,1791.97
471,45,2025-08-14,20.38,1615.68,2183.57
472,45,2025-08-19,31.95,2915.56,4674.00
473,45,2025-08-27,43.85,4071.14,7255.71
474,45,2025-08-29,30.39,2663.04,1964.40
475,45,2025-09-03,11.19,998.32,385.53
476,45,2025-09-30,74.20,5376.42,7340.56


In [33]:
df_combinado.to_excel(r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE SEMANAL\_TEMP_AVANCE_FECHAS.xlsx', index=False, engine='openpyxl', sheet_name='data')
#df_combinado.to_excel('G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/_TEMP_AVANCE_FECHAS.xlsx', index=False, engine='openpyxl', sheet_name='data')

In [34]:
df_combinado.head(2)

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGAS
0,1,2025-06-05,12.28,893.11,815.18
1,1,2025-06-18,65.49,3642.64,4299.72


In [35]:
def cargar_avance_fechas_a_db(df):
    df = df.rename(columns={
            'COD_COS': 'cod_cos',
            'FECHA': 'fecha',
            'AREA': 'area',
            'ESTIMADO': 'estimado',
            'ENTREGAS': 'entregas'
    })
    
    df['cod_cos']     = df['cod_cos'].astype('Int64')
    #df['fecha']   = pd.to_datetime(df["fecha"])
    df['area']  = df['area'].astype('float')
    df['estimado']   = df['estimado'].astype('float')
    df['entregas']   = df['entregas'].astype('float')

    #validad divicion entre 0
    df["tch_estimado"] = np.where(df["area"] > 0,
                              df["estimado"] / df["area"],
                              0)
    #validad divicion entre 0
    df["tch_entregas"] = np.where(df["area"] > 0,
                              df["entregas"] / df["area"],
                              0)
    
    df['diferencia_tn'] = df['entregas'] - df['estimado']
    df["porcen_dif"] = np.where(df["entregas"] > 0,
                              df["diferencia_tn"] / df["entregas"],
                              0)

    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        conn.execute(text(f'TRUNCATE TABLE catastro_iag.data_avance_fechas RESTART IDENTITY'))
        
        # Insertar nuevos datos
        df.to_sql(
            name='data_avance_fechas',
            con=conn,  # conexión cruda dentro de la transacción
            schema='catastro_iag',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de codigos cosecha")

In [36]:
cargar_avance_fechas_a_db(df_combinado)

✅ Se ha actualziado la tabla de codigos cosecha
